In [7]:
import pandas as pd
import random
import os

In [271]:
data_path = r"C:\Users\alexa\Documents\data\wir\project"
fname = r"cc.en.300.bin"

In [272]:
%%time
import fasttext
model = fasttext.load_model(
    os.path.join(data_path, fname))

Wall time: 59 s


In [ ]:
from imgsearch import cosine_sim

def getwv(s):
    return model.get_word_vector(s)

def getwv_multi_labels(labels):
    sublabels = labels.split(', ')
    return sum([getwv(subl) for subl in  sublabels]) / len(sublabels)

get_weight_vec = lambda tup: sum([getwv(label) * weight for weight, label in tup])
get_weight_vec_multuilbl = lambda tup: sum([getwv_multi_labels(label) * weight for weight, label in tup])

In [188]:
%%time
img_emb_dict = dict(zip(
    labelled_df.img_id,
    labelled_df.content.apply(get_weight_vec_multuilbl)
))

Wall time: 1.29 s


In [241]:
import pickle
with open(r"static/img_emb_dict.pickle", "wb") as f:
    pickle.dump(img_emb_dict, f)

In [246]:
def image_search_res5(query):
    
    # get query embedding representation
    q = getwv_multi_labels(query)
    
    if q.sum() != 0:
        
        # parse query
        # ...
    
        # load img emb
        with open(r"static/img_emb_dict.pickle", "rb") as f:
            img_emb_dict = pickle.load(f)
    
        # score and rank images
        score = [cosine_sim(q, img_emb_dict[key]) for key in img_emb_dict]
        score = np.array(score)
        ranked_idx = np.argsort(-score)
        ranked_score = score[ranked_idx]

        # gather images w/ top ranks
        images = labelled_df.loc[ranked_idx, 'image_name'].tolist()[:12]
        
        # output list of images and proba
        return tuple(zip(images, ranked_score))
    else:
        return []

In [269]:
query = 'hiking shoes'
image_search_res5(query)

(('1797-14433-large_default-casque-de-velo-urbain.jpg', 0.7324617),
 ('1681-13966-large_default-affiche-epaule-jete.jpg', 0.7185941),
 ('1736-14042-large_default-roue-du-caviste.jpg', 0.7134589),
 ('413-3333-large_default-whisky-mackmyra-.jpg', 0.71277833),
 ('1179-10038-large_default-livre-nouveaux-artisans.jpg', 0.71108365),
 ('1786-14593-large_default-livre-corpus-tattoo.jpg', 0.71098363),
 ('799-6949-large_default-livre-le-vin-c-est-pas-sorcier.jpg', 0.7073882),
 ('1204-10271-large_default-bougie-pf-candle-co.jpg', 0.70606476),
 ('1017-8642-large_default-atlas-des-constellations.jpg', 0.70595944),
 ('640-5519-large_default-atlas-des-contrees-revees.jpg', 0.7048262),
 ('640-5521-large_default-atlas-des-contrees-revees.jpg', 0.7041713),
 ('1179-10037-large_default-livre-nouveaux-artisans.jpg', 0.70380104))

# run this first

In [59]:
from sklearn.feature_extraction.text import CountVectorizer
from imgsearch import cosine_sim
import numpy as np

In [ ]:
data_path

In [60]:
    path='static/img_top10_labels.csv'
    
    # load image label data
    df = pd.read_csv(path)

    # generate BoW
    vect = CountVectorizer()
    bow = vect.fit_transform(df.content.tolist())
    voc = vect.vocabulary_

In [61]:
#pd.read_csv('static/imagenet_classes_to_labels_map.txt')
with open('static/imagenet_classes_to_labels_map.txt') as f:
    labels = eval(f.read())

In [62]:
def get_topn_labels(row, n=10):
    outdf = row.astype(float).nlargest(n).reset_index()
    outdf.columns = ['label_id','percentage']
    outdf.label_id = outdf.label_id.astype(int)
    outdf['label_name'] = outdf.label_id.map(labels)
    return outdf

In [63]:
def process_img_content(content):
    new_content =  [(w.strip(','), val) for val, sent in content for w in sent.split(' ') ]
    content_dict = {}
    for w in new_content:
        if w[0] in content_dict:
            content_dict[w[0]] += w[1]
        else:
            content_dict[w[0]] = w[1]
    return content_dict

In [240]:
%%time

n_img = 10000

# get weight data
data_path = r"C:\Users\alexa\Documents\data\wir\project\img_proba_vecs_vgg19.csv"
dfw = pd.read_csv(data_path, low_memory=False)

# generate df with descr of top10 classes
labelled_df = dfw.loc[:, dfw.columns[:3]].copy()
labelled_df['content'] = ''

for i, image_name in enumerate(dfw.image_name.loc[:n_img]):
    topnlabels = get_topn_labels(dfw.loc[i, dfw.columns[3:]])
    labelled_df.loc[i, 'content'] = tuple(zip(topnlabels.percentage, topnlabels.label_name)) # keep weight info

# select top n images
labelled_df = labelled_df.loc[:n_img].copy()

Wall time: 31.4 s
